<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Hierarchy

In [ ]:
class Sub extends Component{
  val a = in UInt(8 bits)
  val b = out UInt()    
  b :=  a
}

In [ ]:
class Top extends Component{
  val a = in UInt(8 bits)
  val b = out UInt(8 bits) 
    
  val u_sub = new Sub  //Compile Failed 
  //val u_sub = clockDomain(new Sub) // work well
  u_sub.a := a
  b := u_sub.b
}
showRtl(new Top) 

In [ ]:
object Cxx{
def chain(a: Int, b: Int): Int={ 
    println(s"$a->$b")
    a
  }
}
//val a = chain(1,2)
val d = (1 to 7).reduceRight(chain)
//val c = (1 to 7).reduce(lianjie)//(a,b) => lianjie(a,b)

In [ ]:
class Top extends Component{
  val i = in UInt(16 bits)
  val sel = in UInt(3 bits)
  val o = out UInt(10 bits)
    
  o:= sel.mux(
  0-> i(0 until o.getWidth),
  1-> i(1 until o.getWidth + 1),
  2-> i(2 until o.getWidth + 2),      
  3-> i(3 until o.getWidth + 3),
  default -> i(0 until o.getWidth)
  )
  val tmp = new Array[Tuple2[Int,UInt]](4)
    for ( i <-0 until 10){
        
    }
}
showRtl(new Top)

In [ ]:
class Bmi(aw: Int, dw: Int) extends Bundle with IMasterSlave {
  val rd = Bool()
  val raddr = UInt(aw bits)
  val rdata = UInt(dw bits)
  val wr = Bool()
  val waddr = UInt(aw bits)
  val wdata = UInt(dw bits)

  def cs() = wr || rd
  def addr() = Mux(wr,waddr,raddr)

  override def asMaster(): Unit = {
    in(rdata)
    out(rd,raddr,wr,waddr,wdata)
  }

  def allocate(baseAddr: BigInt, size: BigInt): Bmi = {
    require((baseAddr+size)<=(1<<(aw-1)))
    val ret = new Bmi(log2Up(size),dw)
    val sel = (this.addr < baseAddr + size) && (this.addr >= baseAddr)
    ret.rd := this.rd && sel
    ret.raddr := this.raddr(0 until log2Up(size))
    when(sel){this.rdata := ret.rdata}.otherwise(this.rdata := 0)
    ret.wr := this.wr && sel
    ret.waddr := this.waddr(0 until log2Up(size))
    ret.wdata := this.wdata
    ret
  }
 }

In [ ]:
class Top extends Component{
 val a = slave(new Bmi(8,8))
 val b = master(a.allocate(0,64))
 val c = master(a.allocate(64,64))
}
showRtl(new Top)

In [ ]:
1 to 8
1 until 8 
8 downto 1
val a = (2,3)
a._1

In [ ]:
import spinal.core._
import spinal.lib._
import spinal.lib.bus.misc.SizeMapping   
val  slaves = List(
 1 -> (0x00000000, 1 kB),
 2 -> (0x10000000, 1 kB),
 3 -> (0x20000000, 1 kB)
)
slaves.map(_._2)

In [ ]:
Range(2,9).foldLeft(1)((a,b)=>{println(s"$a-->$b");b})

In [ ]:
class Top extends Component{
  val a = in UInt(8 bits)
  val b = out UInt(8 bits) 
  val c = out UInt(8 bits) 
    
  def pass(x: UInt, n : Int) = {
      val ret = UInt(n bits)
          ret := x 
      ret 
  }
    
  def pass2(x: UInt) = {
      class Fix(n: Int) extends Component {
          val a = in UInt()
          val b = out  UInt() 
          b := pass(in(a), n)
      }
      val res = new Fix(x.getWidth)
      res.a := x
      res.b
  }
    b :=pass2(a)
    c := pass2(a)
}
showRtl(new Top) 

In [ ]:
class Fix(n: Int) extends Component {
    def pass(x: UInt, n : Int) = {
      val ret = UInt(n bits)
          ret := x 
      ret 
     }
    val a = in UInt()
    val b = out  UInt() 
    b := pass(in(a), n)
 }
      
class Top extends Component{
  val a = in UInt(8 bits)
  val b = out UInt(8 bits) 
  val c = out UInt(8 bits) 
 
  def pass2(x: UInt) = {
    val res = new Fix(x.getWidth)
      res.a := x
      res.b
    }
    b := pass2(a)
   val d = pass2(a)
    c := d
}

showRtl(new Top) 

In [ ]:
implicit class BinaryString(sc: StringContext ) {
    def b(args: Any*) = getString(sc)
    
    private def getString(args: Any*): String = {
      val pi   = sc.parts.iterator
      val ai   = args.iterator
      val bldr = new StringBuilder(pi.next().toString)

      while (ai.hasNext) {
        if (ai.hasNext && !ai.next.isInstanceOf[List[_]]) bldr append ai.next
        if (pi.hasNext && !pi.next.isInstanceOf[List[_]]) bldr append pi.next
      }
      bldr.result.replace("_", "")
    }
}

b"10001"

In [ ]:
"10001".toInt(2)